# **Create DataFrame from Nested JSON File in PySpark 3.0 on Colab | Part 5**

In [ ]:
!pwd
!ls
!python --version

In [ ]:
#!wget https://mirrors.estointernet.in/apache/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark

import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Create DataFrame from Nested JSON File in PySpark 3.0").getOrCreate()
print(spark.sparkContext.appName)

Create DataFrame from Nested JSON File in PySpark 3.0


In [22]:
nested_json_file_path = "/content/data/json/sample_nested_json_file.json"

df = spark.read.json(path=nested_json_file_path, multiLine=True)

df.show(10, False)

df.printSchema()

df.select(['CHECK.Check1', 'CHECK.Check2', 'COL', 'DATA', 'IFAM', 'KTM']).show(100, False)

+--------+---+--------------------------------------------------------+----+-------------+
|CHECK   |COL|DATA                                                    |IFAM|KTM          |
+--------+---+--------------------------------------------------------+----+-------------+
|[1, TWO]|21 |[[[[k1, v1], [k2, v2]], 31], [[[k3, v3], [k4, v4]], 33]]|EQR |1548176931466|
+--------+---+--------------------------------------------------------+----+-------------+

root
 |-- CHECK: struct (nullable = true)
 |    |-- Check1: long (nullable = true)
 |    |-- Check2: string (nullable = true)
 |-- COL: long (nullable = true)
 |-- DATA: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Crate: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- key: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- MLrate: string (nullable = true)
 |-- IFAM: string (nullable = true)
 |

In [23]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

def read_nested_json(df):
    column_list = []

    for column_name in df.schema.names:
        print("Outside isinstance loop: " + column_name)
        # Checking column type is ArrayType
        if isinstance(df.schema[column_name].dataType, ArrayType):
            print("Inside isinstance loop of ArrayType: " + column_name)
            df = df.withColumn(column_name, explode(column_name).alias(column_name))
            column_list.append(column_name)

        elif isinstance(df.schema[column_name].dataType, StructType):
            print("Inside isinstance loop of StructType: " + column_name)
            for field in df.schema[column_name].dataType.fields:
                column_list.append(col(column_name + "." + field.name).alias(column_name + "_" + field.name))
        else:
            column_list.append(column_name)

    # Selecting columns using column_list from dataframe: df
    df = df.select(column_list)
    return df

read_nested_json_flag = True

while read_nested_json_flag:
  print("Reading Nested JSON File ... ")
  df = read_nested_json(df)
  df.show(100, False)
  read_nested_json_flag = False

  for column_name in df.schema.names:
    if isinstance(df.schema[column_name].dataType, ArrayType):
      read_nested_json_flag = True
    elif isinstance(df.schema[column_name].dataType, StructType):
      read_nested_json_flag = True

df.show(100, False)


Reading Nested JSON File ... 
Outside isinstance loop: CHECK
Inside isinstance loop of StructType: CHECK
Outside isinstance loop: COL
Outside isinstance loop: DATA
Inside isinstance loop of ArrayType: DATA
Outside isinstance loop: IFAM
Outside isinstance loop: KTM
+------------+------------+---+--------------------------+----+-------------+
|CHECK_Check1|CHECK_Check2|COL|DATA                      |IFAM|KTM          |
+------------+------------+---+--------------------------+----+-------------+
|1           |TWO         |21 |[[[k1, v1], [k2, v2]], 31]|EQR |1548176931466|
|1           |TWO         |21 |[[[k3, v3], [k4, v4]], 33]|EQR |1548176931466|
+------------+------------+---+--------------------------+----+-------------+

Reading Nested JSON File ... 
Outside isinstance loop: CHECK_Check1
Outside isinstance loop: CHECK_Check2
Outside isinstance loop: COL
Outside isinstance loop: DATA
Inside isinstance loop of StructType: DATA
Outside isinstance loop: IFAM
Outside isinstance loop: KTM

In [ ]:
spark.stop()